In [148]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [149]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [150]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [151]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [152]:
# data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
# print(data_pk2_path)
print(media_pk2_path)

/Users/hodung/Desktop/Workspace/silkroad/pk2-extractor/LoadQuestData/data/Media.pk2


In [153]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [154]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [155]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [156]:
_media_file_stream = open(media_pk2_path, 'rb')

In [157]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [158]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [159]:
blowfish_checksum_decoded = "Joymax Pak File"

In [160]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [161]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [162]:
Root = read_blocks_at(256)

In [163]:
blocks_in_memory = {
  "": Root
}

In [164]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [165]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  

def convert_string(value):
    try:
        # First, try to convert to an integer
        return int(value)
    except ValueError:
        try:
            # If it fails, try to convert to a float
            return float(value)
        except ValueError:
            # If both conversions fail, return the original string
            return value


In [166]:
text_data_directory = os.path.join('server_dep', 'silkroad', 'textdata')
text_data_directory

'server_dep/silkroad/textdata'

## Load Text Data

### refqusetreward.txt

In [167]:
refqusetreward_file = os.path.join(text_data_directory, 'refqusetreward.txt')
refqusetreward_file

'server_dep/silkroad/textdata/refqusetreward.txt'

In [168]:
df_refqusetreward = pd.DataFrame(columns=range(25))

lines = get_lines(refqusetreward_file)
for idx, line in enumerate(lines):
  # print(line)
  values = line.split('\t')
  # print(len(values))
  if len(values) < 25:
    continue
  df_refqusetreward.loc[len(df_refqusetreward)] = values
    


In [169]:
df_refqusetreward

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,29,QSP_ALL_POTION_1,0,0,0,0,0,0,0,0,...,xxx,0,0,0,0,xxx,0,xxx,0,xxx
1,38,QNO_KT_SOLDIER_EA2_1,1,1,0,0,0,0,0,0,...,xxx,0,0,0,0,xxx,0,xxx,0,xxx
2,39,QNO_KT_ACCESSORY_1,1,1,0,0,0,0,0,0,...,xxx,0,0,0,0,xxx,0,xxx,0,xxx
3,40,QNO_KT_SMITH_2,1,1,0,0,0,0,0,0,...,xxx,0,0,0,0,xxx,0,xxx,0,xxx
4,41,QNO_KT_ACCESSORY_2,1,1,0,0,0,0,0,0,...,xxx,0,0,0,0,xxx,0,xxx,0,xxx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,1775,QSP_DAILY_LAMP_07,1,1,1,0,0,0,0,0,...,xxx,0,0,0,0,xxx,0,xxx,0,xxx
1254,1776,QSP_DAILY_LAMP_07_CH,1,1,1,0,0,0,0,0,...,xxx,0,0,0,0,xxx,0,xxx,0,xxx
1255,1777,QSP_DAILY_LAMP_07_WC,1,1,1,0,0,0,0,0,...,xxx,0,0,0,0,xxx,0,xxx,0,xxx
1256,1778,QSP_DAILY_LAMP_07_EU,1,1,1,0,0,0,0,0,...,xxx,0,0,0,0,xxx,0,xxx,0,xxx
